# Goblin Freeze & Thaw Demo

This notebook launches a tiny goblin (a pipe-only worker), freezes it with `pdum.criu.goblins`, and thaws clones. By default the heavy CRIU cells are skipped; set `RUN_GOBLIN_DEMO=1` in the environment to run the live demo.

## Environment Check

The next cell verifies prerequisites and prepares helper utilities.

In [ ]:
from __future__ import annotations

import asyncio
import os
import shutil
import signal
import subprocess
import textwrap
import time
from pathlib import Path

from pdum.criu import goblins

BASE_DIR = Path("docs/_goblin_runs")
BASE_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIRS: list[Path] = []
SERVERS = []
FREEZE_DIR: Path | None = None

COUNTER_SCRIPT = textwrap.dedent(
    """
    import os
    import sys

    os.setsid()
    counter = 0
    print("ready", flush=True)

    for line in sys.stdin:
        if not line:
            break
        counter += 1
        sys.stdout.write(f"{counter}:{line}")
        sys.stdout.flush()
    """
)

def fresh_run_dir(label: str) -> Path:
    path = BASE_DIR / f"{label}-{int(time.time() * 1000)}"
    path.mkdir(parents=True, exist_ok=True)
    RUN_DIRS.append(path)
    return path

def check_prereqs() -> tuple[bool, str]:
    if os.environ.get("RUN_GOBLIN_DEMO") != "1":
        return False, "Set RUN_GOBLIN_DEMO=1 to execute the live steps."
    if os.name != "posix":
        return False, "CRIU requires Linux"
    for cmd in ("criu", "sudo", "pgrep"):
        if shutil.which(cmd) is None:
            return False, f"{cmd} is not available in PATH"
    result = subprocess.run(["sudo", "-n", "true"], capture_output=True)
    if result.returncode != 0:
        detail = (result.stderr or result.stdout).decode("utf-8", errors="ignore").strip()
        return False, f"password-less sudo is required ({detail})"
    return True, "Ready"

LIVE_READY, LIVE_REASON = check_prereqs()
print(LIVE_REASON)


In [ ]:
class GoblinServer:
    def __init__(self, run_dir: Path):
        self.run_dir = run_dir
        self.proc = subprocess.Popen(
            [os.environ.get("PYTHON", "python3"), "-u", "-c", COUNTER_SCRIPT],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )
        SERVERS.append(self.proc)
        self.proc.stdout.readline()

    def stop(self) -> None:
        if self.proc.poll() is None:
            self.proc.terminate()
            try:
                self.proc.wait(timeout=3)
            except subprocess.TimeoutExpired:
                self.proc.kill()


class GoblinClient:
    def __init__(self, stdin, stdout):
        self.stdin = stdin
        self.stdout = stdout

    def send(self, text: str) -> str:
        payload = text.rstrip("\n") + "\n"
        self.stdin.write(payload.encode("utf-8"))
        self.stdin.flush()
        return self.stdout.readline().decode("utf-8").strip()


In [ ]:
if LIVE_READY:
    run_dir = fresh_run_dir("server")
    server = GoblinServer(run_dir)
    client = GoblinClient(server.proc.stdin, server.proc.stdout)
    print(client.send("alpha"))
    print(client.send("beta"))
else:
    server = None
    print(f"Skipping launch: {LIVE_REASON}")


In [ ]:
if LIVE_READY and server is not None:
    FREEZE_DIR = fresh_run_dir("freeze")
    log_path = goblins.freeze(server.proc.pid, FREEZE_DIR, leave_running=False)
    print(f"Frozen goblin into {FREEZE_DIR}\nLog: {log_path}")
    server.stop()
else:
    print(f"Skipping freeze: {LIVE_REASON}")


In [ ]:
if LIVE_READY and FREEZE_DIR:
    thawed = goblins.thaw(FREEZE_DIR)
    clone_client = GoblinClient(thawed.stdin, thawed.stdout)
    print(clone_client.send("gamma"))
    print(clone_client.send("delta"))
    os.kill(thawed.pid, signal.SIGTERM)
    thawed.stdin.close()
    thawed.stdout.close()
    thawed.stderr.close()
else:
    print(f"Skipping sync thaw: {LIVE_REASON}")


In [ ]:
if LIVE_READY and FREEZE_DIR:
    async def thaw_async_demo():
        handle = await goblins.thaw_async(FREEZE_DIR)
        try:
            handle.stdin.write("epsilon\n")
            await handle.stdin.drain()
            response = await handle.stdout.readline()
            print(response.decode().strip())
        finally:
            os.kill(handle.pid, signal.SIGTERM)
            await handle.close()

    asyncio.run(thaw_async_demo())
else:
    print(f"Skipping async thaw: {LIVE_REASON}")


In [ ]:
for proc in SERVERS:
    if proc.poll() is None:
        proc.terminate()
        try:
            proc.wait(timeout=3)
        except subprocess.TimeoutExpired:
            proc.kill()

for path in RUN_DIRS:
    shutil.rmtree(path, ignore_errors=True)
print("Cleaned up goblin runs.")
